In [98]:
# ============================================================================
# 0. SETUP INSTRUCTIONS
# ============================================================================
"""
BEFORE RUNNING THIS SCRIPT:

1. INSTALL/UPGRADE OpenAI SDK (Use chat.completions API):
   pip install --upgrade openai

2. Install all dependencies:
   pip install langgraph openai python-dotenv

3. Set your OpenAI API key:
   
   OPTION A (Recommended - Temporary):
   export OPENAI_API_KEY="sk-proj-your-actual-key-here"  # Linux/Mac
   set OPENAI_API_KEY=sk-proj-your-actual-key-here       # Windows
   
   OPTION B (Permanent - Using .env file):
   Create a .env file in the same directory with:
   OPENAI_API_KEY=sk-proj-your-actual-key-here

MODEL BEING USED:
- gpt-3.5-turbo: Low cost (~$0.001 per 1K tokens), fast, good for most tasks

TROUBLESHOOTING:
- If you see "'OpenAI' object has no attribute 'messages'" error,
  this code uses chat.completions.create() which is compatible
- Make sure your API key is valid and has credits
"""

# Uncomment these lines if using .env file:
# from dotenv import load_dotenv
# load_dotenv()

from langgraph.graph import StateGraph, START, END
from typing import TypedDict, Annotated
from openai import OpenAI
import json
from operator import add
import os
import sys

# ============================================================================
# VERSION CHECK
# ============================================================================
try:
    import openai
    print(f"✓ OpenAI SDK version: {openai.__version__}")
    
    # Check if version is high enough (need 1.0.0+)
    version_parts = openai.__version__.split('.')
    major_version = int(version_parts[0])
    
    if major_version < 1:
        print("\n❌ ERROR: Your OpenAI SDK is too old!")
        print("Please upgrade with:")
        print("  pip install --upgrade openai")
        print("\nExiting...")
        sys.exit(1)
except Exception as e:
    print(f"⚠️  Could not verify OpenAI version: {e}")
    print("Make sure you have openai 1.0.0 or higher installed")
    print("Install with: pip install --upgrade openai\n")

# ============================================================================
# 1. STATE DEFINITION - Using Annotated for reducer functions
# ============================================================================
class AgentState(TypedDict):
    """
    State tracks:
    - input: user query
    - steps: track workflow progression
    - documents: retrieved knowledge
    - reasoning: LLM thinking process
    - final_answer: output
    """
    input: str
    steps: Annotated[list, add]  # Accumulate steps
    documents: list
    reasoning: str
    final_answer: str

# ============================================================================
# 2. INITIALIZE OpenAI CLIENT
# ============================================================================
# Make sure your OPENAI_API_KEY environment variable is set
api_key = os.getenv("OPENAI_API_KEY")

if not api_key:
    print("\n" + "="*70)
    print("❌ ERROR: OPENAI_API_KEY not found!")
    print("="*70)
    print("\nTo fix this, set your API key:")
    print("\nLinux/Mac:")
    print('  export OPENAI_API_KEY="sk-proj-your-key-here"')
    print("\nWindows (PowerShell):")
    print('  $env:OPENAI_API_KEY="sk-proj-your-key-here"')
    print("\nWindows (Command Prompt):")
    print('  set OPENAI_API_KEY=sk-proj-your-key-here')
    print("\nOr create a .env file with:")
    print('  OPENAI_API_KEY=sk-proj-your-key-here')
    print("="*70 + "\n")
    sys.exit(1)

# Initialize client with proper error handling
try:
    client = OpenAI(api_key=api_key)
    # Test the connection
    print("✓ OpenAI client initialized successfully")
    print(f"✓ API Key is set (length: {len(api_key)} characters)\n")
except Exception as e:
    print(f"❌ Failed to initialize OpenAI client: {e}")
    sys.exit(1)

# ============================================================================
# 3. SIMPLE KNOWLEDGE BASE - Mock retrieval
# ============================================================================
KNOWLEDGE_BASE = {
    "python": [
        "Python is a high-level programming language",
        "It uses indentation for code blocks",
        "Popular frameworks: Django, Flask, FastAPI"
    ],
    "api": [
        "API stands for Application Programming Interface",
        "REST APIs use HTTP methods: GET, POST, PUT, DELETE",
        "APIs return JSON or XML responses"
    ],
    "database": [
        "Databases store structured data",
        "SQL is used for relational databases",
        "NoSQL databases include MongoDB, Redis"
    ]
}

# ============================================================================
# 4. WORKFLOW NODES - Simple sequential steps
# ============================================================================

def retrieve_documents(state: AgentState) -> AgentState:
    """Step 1: Retrieve relevant documents from knowledge base"""
    print("\n📚 Step 1: Retrieving documents...")
    
    query = state["input"].lower()
    docs = []
    
    # Simple keyword matching
    for topic, content in KNOWLEDGE_BASE.items():
        if topic in query:
            docs.extend(content)
    
    if not docs:
        docs = ["General knowledge: No specific documents found"]
    
    state["steps"].append("✓ Retrieved documents")
    state["documents"] = docs
    print(f"   Found {len(docs)} documents")
    
    return state

def generate_reasoning(state: AgentState) -> AgentState:
    """Step 2: Generate reasoning using OpenAI"""
    print("\n🧠 Step 2: Generating reasoning...")
    
    documents_text = "\n".join(state["documents"])
    
    try:
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",  # Low cost model
            max_tokens=200,
            messages=[{
                "role": "user",
                "content": f"""Given these documents, explain how they relate to the question:
                
Question: {state['input']}

Documents:
{documents_text}

Provide a brief reasoning in 2-3 sentences."""
            }]
        )
        
        reasoning = response.choices[0].message.content
        state["reasoning"] = reasoning
        state["steps"].append("✓ Generated reasoning")
        print(f"   Reasoning: {reasoning[:100]}...")
        
    except Exception as e:
        print(f"   ❌ Error: {e}")
        print("   Using fallback reasoning...")
        state["reasoning"] = f"Based on the documents provided, the query '{state['input']}' relates to the retrieved information."
        state["steps"].append("✓ Generated reasoning (fallback)")
    
    return state

def generate_answer(state: AgentState) -> AgentState:
    """Step 3: Generate final answer using retrieved context and reasoning"""
    print("\n💡 Step 3: Generating final answer...")
    
    documents_text = "\n".join(state["documents"])
    
    try:
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",  # Low cost model
            max_tokens=300,
            messages=[{
                "role": "user",
                "content": f"""Based on the following:

Question: {state['input']}

Knowledge:
{documents_text}

Reasoning: {state['reasoning']}

Provide a clear, concise answer."""
            }]
        )
        
        answer = response.choices[0].message.content
        state["final_answer"] = answer
        state["steps"].append("✓ Generated final answer")
        print(f"   Generated answer")
        
    except Exception as e:
        print(f"   ❌ Error: {e}")
        state["final_answer"] = f"I couldn't generate a full answer, but based on the documents: {', '.join(state['documents'][:2])}"
        state["steps"].append("✓ Generated final answer (fallback)")
    
    return state

def validate_answer(state: AgentState) -> AgentState:
    """Step 4: Validate answer quality using LLM"""
    print("\n✅ Step 4: Validating answer...")
    
    try:
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",  # Low cost model
            max_tokens=100,
            messages=[{
                "role": "user",
                "content": f"""Is this answer helpful and accurate? Answer with YES or NO only:

Question: {state['input']}
Answer: {state['final_answer']}"""
            }]
        )
        
        validation = response.choices[0].message.content.strip()
        state["steps"].append(f"✓ Validated answer ({validation})")
        print(f"   Validation: {validation}")
        
    except Exception as e:
        print(f"   ❌ Error during validation: {e}")
        state["steps"].append("✓ Validated answer (skipped)")
    
    return state

# ============================================================================
# 5. BUILD THE GRAPH
# ============================================================================
workflow = StateGraph(AgentState)

# Add nodes in sequence
workflow.add_node("retrieve", retrieve_documents)
workflow.add_node("reasoning", generate_reasoning)
workflow.add_node("answer", generate_answer)
workflow.add_node("validate", validate_answer)

# Create linear workflow
workflow.add_edge(START, "retrieve")
workflow.add_edge("retrieve", "reasoning")
workflow.add_edge("reasoning", "answer")
workflow.add_edge("validate", END)
workflow.add_edge("answer", "validate")

# Compile
graph = workflow.compile()

# ============================================================================
# 6. RUN THE AGENT
# ============================================================================
def run_agent(query: str):
    """Execute the workflow"""
    
    initial_state = AgentState(
        input=query,
        steps=[],
        documents=[],
        reasoning="",
        final_answer=""
    )
    
    print("\n" + "="*70)
    print(f"🚀 LANGGRAPH AGENT STARTED")
    print(f"Query: {query}")
    print("="*70)
    
    # Invoke the graph
    final_state = graph.invoke(initial_state)
    print("!111111111111111111111111111111111111111111")
    print(final_state)
    print("1111111111111111111111111111111111111111111")
    
    # Display results
    print("\n" + "="*70)
    print("📊 WORKFLOW EXECUTION:")
    print("="*70)
    for step in final_state["steps"]:
        print(f"  {step}")
    
    print("\n" + "="*70)
    print("📄 DOCUMENTS RETRIEVED:")
    print("="*70)
    for doc in final_state["documents"]:
        print(f"  • {doc}")
    
    print("\n" + "="*70)
    print("🧠 REASONING:")
    print("="*70)
    print(final_state["reasoning"])
    
    print("\n" + "="*70)
    print("✨ FINAL ANSWER:")
    print("="*70)
    print(final_state["final_answer"])
    print("="*70 + "\n")
    
    return final_state

# ============================================================================
# 7. TEST THE AGENT
# ============================================================================
if __name__ == "__main__":
    # Test queries
    queries = [
        "What is Python and why is it useful?",
        # "Explain REST APIs to me",
        # "Tell me about databases"
    ]
    
    for query in queries:
        run_agent(query)

✓ OpenAI SDK version: 2.5.0
✓ OpenAI client initialized successfully
✓ API Key is set (length: 164 characters)


🚀 LANGGRAPH AGENT STARTED
Query: What is Python and why is it useful?

📚 Step 1: Retrieving documents...
   Found 3 documents

🧠 Step 2: Generating reasoning...
   Reasoning: Python is a high-level programming language known for its readability and simplicity, making it usef...

💡 Step 3: Generating final answer...
   Generated answer

✅ Step 4: Validating answer...
   Validation: YES
!111111111111111111111111111111111111111111
{'input': 'What is Python and why is it useful?', 'steps': ['✓ Retrieved documents', '✓ Retrieved documents', '✓ Generated reasoning', '✓ Retrieved documents', '✓ Retrieved documents', '✓ Generated reasoning', '✓ Generated final answer', '✓ Retrieved documents', '✓ Retrieved documents', '✓ Generated reasoning', '✓ Retrieved documents', '✓ Retrieved documents', '✓ Generated reasoning', '✓ Generated final answer', '✓ Validated answer (YES)', '✓ Retrieve